<a href="https://colab.research.google.com/github/MSiswanto/MachineLearning/blob/main/infrared_images_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install

In [ ]:
!pip install --upgrade pip
#!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [ ]:
!pip install opencv-python-headless

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.19.3
    Uninstalling scikit-image-0.19.3:
      Successfully uninstalled scikit-image-0.19.3



Importing Library

In [ ]:
import os      
import imutils
import cv2
import time
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

from PIL import Image

from skimage.feature import hog
from skimage import io
from skimage.color import rgb2gray

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

Resizing Images

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
#from google.colab import auth
#auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_image(row_id, root= "/content/drive/MyDrive/images"):
    filename = "image{:04d}.jpg".format(row_id)
    file_path = os.path.join(root, filename)
    imag = Image.open(file_path)
    img = imag.resize((64, 128))
    return np.array(img)

In [ ]:
# Load image using the get_image function
img = get_image(row_id=1)

# Display image using cv2_imshow
cv2_imshow(img)

Extracting Image HOG Features

In [ ]:
def create_features(img):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd


In [ ]:
from skimage.color import rgb2gray

def create_features(img):                                                 
    
    color_features = img.flatten()                                                    # flatten three channel color image
    grey_image = rgb2gray(img)                                                        # convert image to greyscale
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(2,2))        # get HOG features from greyscale image
    flat_features = np.hstack(hog_features)                                           # combine color and hog features into a single array

    return flat_features

Creating HOG Features Matrix for Training Dataset

In [ ]:
def create_feature_matrix(label_dataframe): 
    
    c=1
    features_list = []                                   # List of Features of an Image
    for img_id in label_dataframe.index:
        img = get_image(img_id)                          # load image
        image_features = create_features(img)            # get features for image
        features_list.append(image_features)
        print(c)                                         # Displays the count of Images converted to Vector
        c+=1
    feature_matrix = np.array(features_list)             # convert list of arrays into a matrix
    return feature_matrix

PCA to reduce Dimensionality of Feature Matrix

In [ ]:
def pcca(feature_matrix):
                                   
    print('Feature matrix shape is: ', feature_matrix.shape)           # get shape of feature matrix
    ss = StandardScaler()                                              # define standard scaler
    human_stand = ss.fit_transform(feature_matrix)                     # run this on our feature matrix
    pca = PCA(n_components=50)                                         # set the number of components wisely, maybe less than 75
    human_pca = pca.fit_transform(human_stand)                         # use fit_transform to run PCA on our standardized matrix
    print('PCA matrix shape is: ', human_pca.shape)                    # look at new shape
    return human_pca

Sliding Window Mechanism running throughout an Image

In [ ]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0] - windowSize[1], stepSize):
        for x in range(0, image.shape[1] - windowSize[0], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [ ]:
#def sliding_window(image, stepSize, windowSize):
    
#    for y in range(0, image.shape[0], stepSize):                                  # slide a window across the image
#        for x in range(0, image.shape[1], stepSize):                              # yield the current window
#            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
                    

Non-Maximum Suppression for removing False Positive Regions

In [ ]:
def non_max_suppression_fast(boxes, overlapThresh):
    
    if len(boxes) == 0:                                                  # if there are no boxes, return an empty list
        return []
    if boxes.dtype.kind == "i":                                          # if the bounding boxes integers, convert them to floats --
        boxes = boxes.astype("float")                                    # this is important since we'll be doing a bunch of divisions
    pick = []                                                            # initialize the list of picked indexes
    x1 = boxes[:,0]                                                      # grab the coordinates of the bounding boxes
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)                                 # compute the area of the bounding boxes 
    idxs = np.argsort(y2)                                                # sort the bounding boxes by the bottom-right y-coordinate of the bounding box
    while len(idxs) > 0:                                                 # keep looping while some indexes still remain in the indexes list
        last = len(idxs) - 1                                             # grab the last index in the indexes list  
        i = idxs[last]                                                   
        pick.append(i)                                                   # add the index value to the list of picked indexes
        xx1 = np.maximum(x1[i], x1[idxs[:last]])                         # find the largest (x, y) coordinates for the start of the bounding box
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])                         # find the smallest (x, y) coordinates for the end of the bounding box
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)                                 # compute the width of the bounding box
        h = np.maximum(0, yy2 - yy1 + 1)                                 # compute the height of the bounding box
        overlap = (w * h) / area[idxs[:last]]                            # compute the ratio of overlap
        idxs = np.delete(idxs, np.concatenate(([last],                   # delete all indexes from the index list that have
            np.where(overlap > overlapThresh)[0])))
    return boxes[pick].astype("int")                                     # return only the bounding boxes that were picked using the integer data type

Training Procedure for Image Dataset, using Support Vector Classifier

In [ ]:
def create_feature_matrix(labels, img_dir):
    feature_matrix = []
    for i in range(len(labels)):
        img_path = os.path.join(img_dir, labels.index[i])
        img = cv2.imread(img_path)
        feature_vector = np.array([cv2.mean(img)[:3]])
        feature_matrix.append(feature_vector)
    return np.vstack(feature_matrix)


In [ ]:
import pandas as pd

# Load image labels from a CSV file
labels = pd.read_csv("/content/drive/MyDrive/labels.csv", index_col=0)  #/content/sample_data/_annotations.csv", index_col=0)

# Set the directory where the image files are stored
img_dir = "/content/drive/MyDrive/images" #/content/drive/MyDrive/images"

# Create a feature matrix from the images
feature_matrix = create_feature_matrix(labels, img_dir)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC

# Generate random feature matrix with 100 samples and 10 features
feature_matrix = np.random.randn(500, 2266)

# Generate a small feature matrix with only 3 samples and 2 features
#feature_matrix = np.array([[1, 2], [3, 4], [5, 6]])

# Try to perform PCA with n_components=50
try:
    pca = PCA(n_components=50)
    X_pca = pca.fit_transform(feature_matrix)
except ValueError as e:
    # Handle the error by reducing the number of components
    print(f"Error: {e}")
    max_components = min(feature_matrix.shape)
    pca = PCA(n_components=max_components)
    X_pca = pca.fit_transform(feature_matrix)

    # Perform principal component analysis  
    X_train = pca.fit_transform(feature_matrix)
    X_train = pd.DataFrame(X_train)

    # Access the labels of training images
    y_train = pd.Series(labels.label.values)

    # Train a support vector classifier
    svm = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)
    svm.fit(X_train, y_train)

    # Look at the distribution of labels in the train set
    pd.Series(y_train).value_counts()
    
print(f"Original shape: {feature_matrix.shape}")
print(f"Reduced shape: {X_pca.shape}")


Original shape: (500, 2266)
Reduced shape: (500, 50)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Reshape y to have shape of (n,)
y = np.array(1 if i in positive_indexes else 0 for i in range(len(t))).reshape(-1, 1)
#y = y.ravel()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an SVM classifier
svm = SVC(kernel='linear', probability=True)  #svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)

# Evaluate the classifier
accuracy = svm.score(X_test, y_test)
print("Accuracy:", accuracy)


Applying Trained Model to Detect Human in a Test Image


In [ ]:
import cv2

def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap between the computed
        # bounding box and the bounding boxes in the boxes list,
        # and update the indexes list
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")


In [ ]:
import time
from imutils.object_detection import non_max_suppression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

from skimage.color import rgb2gray

In [ ]:
from google.colab.patches import cv2_imshow

image = cv2.imread("/content/drive/MyDrive/images/image0001.jpg")    
#image = cv2.imread(r"/content/drive/MyDrive/dataset_irimage/")    

scale_percent = 200                                                                         # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)                   
                                                                                            # New dimensions of Image Window
image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)  

(winW , WinH) = (64 , 128)    
winH = 128                                                              # Dimensions of Sliding Window
t=[]                                                                                        # Feature Matrix for test image portions
k=[]       
                                                                                            # Indexes corresponding to image portions
for (x, y, window) in sliding_window(image, stepSize=16, windowSize=(winW, winH)):          # loop over the sliding window for each layer of the pyramid
    if window.shape[0] != winH or window.shape[1] != winW:                                  # if the window does not meet our desired window size, ignore it
        continue           

    clone = image.copy()
    op=clone[y:y + winH, x:x + winW]
    f = create_features(op)
    t.append(f)
    k.append([x,y])
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
    cv2_imshow(clone) #("Window", clone)
    cv2.waitKey(1)
    time.sleep(0.025)
cv2.destroyAllWindows()

# THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE WINDOW
# since we do not have a classifier, we'll just draw the window

hpp=pcca(t)
x_p=pd.DataFrame(hpp)
y_p=svm.predict(x_p)
pp=list(y_p)                                                                                # Predict the portions where human is detected
clon = image.copy()
nn=[]                                                                                       # List of indexes of Human Detection Portions
for i in range(0,len(pp)):
    if pp[i]==1.0:
        x=k[i][0]
        y=k[i][1]
        nn.append([x,y,x+winW,y+winH])
        cv2.rectangle(clon, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
        cv2.imshow("Window", clon)
        cv2.waitKey(50)
        time.sleep(0.025)
cv2.destroyAllWindows()

ll=non_max_suppression_fast(np.array(nn),0.4)                                              # Applied Non-Max Suppression on Human Detected portions
l=list(ll)                                                                                 # List of Indexes of Actual Human Regions 
clo=image.copy()
for kl in l:
    cv2.rectangle(clo, (kl[0], kl[1]), (kl[2],kl[3]), (0, 255, 0), 1)                      # Display the most probable Detection Regions out of False Positives
    cv2_imshow(clo)
    cv2.waitKey(0)
    time.sleep(0.025)
cv2.destroyAllWindows()